In [1]:
import urllib.request, json 
import pandas as pd
with urllib.request.urlopen("https://www.datos.gov.co/resource/fbht-2fzd.json") as url:
    df = pd.read_json(url)
df

,fecha_despacho,anio_despacho,mes_despacho,dia_despacho,tipo_proveedor,codigo_municipio_dane,departamento_proveedor,municipio_proveedor,tipo_comprador,zona_frontera,subtipo_comprador,codigo_municipio_dane_destino,departamento,municipio,producto,volumen_despachado,latitud,longitud
0,2023-02-09,2023,2,9,DISTRIBUIDOR MAYORISTA,68190,SANTANDER,CIMITARRA,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,15572,BOYACA,PUERTO BOYACA,DIESEL,3340,6.320682,-73.952583
1,2021-03-21,2021,3,21,DISTRIBUIDOR MAYORISTA,5001,ANTIOQUIA,MEDELLIN,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,5001,ANTIOQUIA,MEDELLIN,CORRIENTE,3823,5.610110,-74.886818
2,2022-10-25,2022,10,25,DISTRIBUIDOR MAYORISTA,8078,ATLANTICO,BARANOA,DISTRIBUIDOR MINORISTA,SI,ESTACION DE SERVICIO AUTOMOTRIZ,20001,CESAR,VALLEDUPAR,DIESEL,3370,10.794881,-74.916100
3,2021-12-11,2021,12,11,DISTRIBUIDOR MAYORISTA,68001,SANTANDER,BUCARAMANGA,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,68547,SANTANDER,PIEDECUESTA,CORRIENTE,6000,7.116267,-73.132599
4,2021-03-11,2021,3,11,DISTRIBUIDOR MAYORISTA,68092,SANTANDER,BETULIA,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,68679,SANTANDER,SAN GIL,CORRIENTE,2000,6.899525,-73.283669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-01-07,2022,1,7,DISTRIBUIDOR MAYORISTA,18001,CAQUETA,FLORENCIA,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,18001,CAQUETA,FLORENCIA,CORRIENTE,3415,1.617923,-75.608559
996,2023-05-10,2023,5,10,DISTRIBUIDOR MAYORISTA,8078,ATLANTICO,BARANOA,DISTRIBUIDOR MINORISTA,SI,ESTACION DE SERVICIO AUTOMOTRIZ,20238,CESAR,EL COPEY,DIESEL,11250,10.794881,-74.916100
997,2021-06-24,2021,6,24,DISTRIBUIDOR MAYORISTA,68081,SANTANDER,BARRANCABERMEJA,DISTRIBUIDOR MINORISTA,NO,ESTACION DE SERVICIO AUTOMOTRIZ,68575,SANTANDER,PUERTO WILCHES,CORRIENTE,2040,7.064964,-73.849960
998,2021-02-05,2021,2,5,DISTRIBUIDOR MAYORISTA,54874,NORTE DE SANTANDER,VILLA DEL ROSARIO,DISTRIBUIDOR MINORISTA,SI,ESTACION DE SERVICIO AUTOMOTRIZ,54001,NORTE DE SANTANDER,SAN JOSE DE CUCUTA,CORRIENTE,2050,7.847487,-72.470070


In [2]:
df2 = df[['departamento','volumen_despachado']]
df3 = df2.groupby(['departamento'])['volumen_despachado'].sum().to_frame()
df3

,volumen_despachado
departamento,
AMAZONAS,17596
ANTIOQUIA,503626
ARAUCA,19552
ATLANTICO,121016
BOGOTA D.C.,381616
BOLIVAR,87895
BOYACA,82984
CALDAS,40773
CAQUETA,20780


In [5]:
import pandas as pd
import numpy as np
import plotly.express as px

df = df[['volumen_despachado', 'latitud', 'longitud', 'departamento_proveedor']]


fig = px.density_mapbox(df, lat='latitud', lon='longitud', z='volumen_despachado',
                        mapbox_style="stamen-terrain", width=1200, height=1200)
 
fig


In [7]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
import requests
from pathlib import Path
from zipfile import ZipFile
import os
app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4("Gas"),
        html.P(
            "px.scatter_geo is used to plot points on globe across geolocations while "
            "px.scatter_mapbox is used to plot points on map across geolocations."
        ),
        dcc.Graph(id="graph"),
        html.P(
            "Enter the type of graph you want to plot: (scatter_geo or scatter_mapbox)"
        ),
        dcc.Dropdown(
            id="type",
            options=["density_mapbox", "scatter_geo", "departamentos"],
            value="departamentos",
            clearable=False,
        ),
    ]
)


@app.callback(
    Output("graph", "figure"),
    Input("type", "value"),
)
def generate_chart(values):
    
    if values == "density_mapbox":
        # changing so that the default position will be US
        fig = px.density_mapbox(df, lat='latitud', lon='longitud', z='volumen_despachado', zoom=3,
                        mapbox_style="stamen-terrain")

        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    elif values == "departamentos":
        
        geodf = gpd.read_file('C:/Users/laco-/Documents/Notebooks/MGN_DPTO_POLITICO.shp')
        #geodf["rand"] = np.random.randint(1, 100, len(geodf))
        # shape file is a different CRS,  change to lon/lat GPS co-ordinates
        geodf = geodf.to_crs("WGS84")
        geodf = geodf[['DPTO_CNMBR', 'geometry']]
        geodf= geodf.rename(columns={"DPTO_CNMBR": "departamento"})
        geodf = geodf.merge(df3,on='departamento')
        
        fig = px.choropleth_mapbox(
        geodf.set_index("departamento"),
        geojson=geodf.geometry,
        locations=geodf.index,
        color="volumen_despachado",
        center=dict(lat=4.11, lon=-74.3),
        mapbox_style="open-street-map",
        zoom=3,)
    
    
       
    

    else:
        fig = px.scatter_mapbox(
                    df, lat='latitud', lon='longitud', size='volumen_despachado',
                    zoom=3,)
                
        fig.update_layout(mapbox_style="open-street-map")
    return fig


if __name__ == "__main__":
    app.run_server(debug=True)
    #app.run_server(host='127.0.0.1', port=8088, debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

In [ ]:

geodf = gpd.read_file('C:/Users/laco-/Documents/Notebooks/MGN_DPTO_POLITICO.shp')
#geodf["rand"] = np.random.randint(1, 100, len(geodf))
geodf = geodf.to_crs("WGS84")
#geodf
geodf = geodf[['DPTO_CNMBR', 'geometry']]



In [ ]:
geodf= geodf.rename(columns={"DPTO_CNMBR": "departamento"})
geodf = geodf.merge(df3,on='departamento')
geodf